# Oil Price Prediction Using Military Flight Data

First, let's check our environment setup and dependencies.

In [ ]:
import sys
import os
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(''))))

import pandas as pd
import numpy as np
from pathlib import Path
import logging
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go

from src.ml_models.predictor_model import OilPricePredictor
from src.ml_models.feature_engineering import FeatureEngineer
from src.data_collection.oil_price_collector import OilPriceCollector
from src.data_collection.flightradar_scraper import FlightRadarScraper
from src.data_collection.base_monitor import BaseMonitor
from src.utils.logging_config import setup_logging

setup_logging()

Missing packages: selenium

Install using:
pip install selenium


ImportError: Missing required packages

In [ ]:
import os
import sys
from pathlib import Path

# Get the absolute path to the project root
notebook_dir = Path().absolute()
project_root = notebook_dir.parent

# Add project root to Python path if not already there
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Verify the path is set correctly
print(f"Project root: {project_root}")
print(f"Python path: {sys.path[0]}")

# Now import project modules
try:
    from src.ml_models.predictor_model import OilPricePredictor
    from src.ml_models.feature_engineering import FeatureEngineer
    from src.data_collection.oil_price_collector import OilPriceCollector
    from src.data_collection.flightradar_scraper import FlightRadarScraper
    from src.data_collection.base_monitor import BaseMonitor
    print("All project modules imported successfully!")
except ImportError as e:
    print(f"Error importing project modules: {e}")
    print("\nMake sure you're running this notebook from the project root directory")
    raise

import logging
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go

from src.utils.logging_config import setup_logging

setup_logging()

## 1. Load Latest Trained Model

In [ ]:
def load_latest_model():
    model_files = list(Path("../data/models").glob("oil_predictor_*.pkl"))
    if not model_files:
        raise FileNotFoundError("No trained models found")
    
    latest_model = max(model_files, key=os.path.getctime)
    print(f"Loading model: {latest_model}")
    
    predictor = OilPricePredictor()
    predictor.load_model(latest_model)
    
    return predictor

predictor = load_latest_model()

## 2. Collect data

In [ ]:
oil_collector = OilPriceCollector()
flight_scraper = FlightRadarScraper()
base_monitor = BaseMonitor()

try:
    print("Fetching current oil prices...")
    current_prices = oil_collector.fetch_current_prices()
    print("Fetching historical oil data...")
    recent_oil = oil_collector.fetch_historical_data(days=7)
    
    print("Fetching flight data...")
    flights = flight_scraper.get_middle_east_flights()
    
    if flights:
        flight_df = pd.DataFrame(flights)
        flight_df['is_military'] = flight_df['callsign'].apply(base_monitor.is_military_aircraft)

        base_info = []
        for _, flight in flight_df.iterrows():
            is_near, base_name = base_monitor.is_near_base(flight['latitude'], flight['longitude'])
            base_info.append({
                'is_near_base': is_near,
                'base_name': base_name if is_near else 'None'
            })
        
        base_df = pd.DataFrame(base_info)
        flight_df = pd.concat([flight_df, base_df], axis=1)
finally:
    flight_scraper.close_driver()

print("\nCurrent Oil Prices:")
print(f"Brent Crude: ${current_prices.get('BZ=F', 'N/A')}")
print(f"WTI Crude: ${current_prices.get('CL=F', 'N/A')}")

## 3. Process and Visualise Flight Data

In [ ]:
if not flight_df.empty:
    military_count = flight_df['is_military'].sum()
    total_flights = len(flight_df)
    
    # Create pie chart of military vs civilian flights
    labels = ['Military', 'Civilian']
    values = [military_count, total_flights - military_count]
    
    fig = go.Figure(data=[go.Pie(labels=labels, values=values)])
    fig.update_layout(title='Military vs Civilian Flights')
    fig.show()
    
    # Create scatter map of flights
    fig = px.scatter_mapbox(flight_df,
                           lat='latitude',
                           lon='longitude',
                           color='is_military',
                           hover_data=['callsign', 'base_name'],
                           zoom=4,
                           title='Flight Locations')
    fig.update_layout(mapbox_style='open-street-map')
    fig.show()

## 4. Feature Engineering and Prediction

In [ ]:
feature_engineer = FeatureEngineer()

if not flight_df.empty:
    flight_features = feature_engineer.create_flight_features(flight_df)
else:
    print("No flight data available")
    flight_features = pd.DataFrame()

oil_features = feature_engineer.create_oil_features(recent_oil)

if not flight_features.empty:
    combined_features = feature_engineer.combine_features(flight_features, oil_features)
else:
    combined_features = oil_features
    print("Prediction based only on oil price features")

latest_features = combined_features.iloc[-1:].values
prediction = predictor.predict(latest_features)[0]

print(f"\nPredicted Brent Crude Price: ${prediction:.2f}")

## 5. Save Prediction Results

In [ ]:
prediction_data = {
    'timestamp': datetime.now().isoformat(),
    'predicted_price': float(prediction),
    'current_bz_price': current_prices.get('BZ=F'),
    'current_cl_price': current_prices.get('CL=F'),
    'military_flights': int(flight_df['is_military'].sum()) if not flight_df.empty else 0,
    'total_flights': len(flight_df) if not flight_df.empty else 0,
    'model_used': predictor.best_model_name
}

Path("../data/predictions").mkdir(parents=True, exist_ok=True)
prediction_file = f"../data/predictions/prediction_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"

import json
with open(prediction_file, 'w') as f:
    json.dump(prediction_data, f, indent=2)

print(f"Prediction saved to {prediction_file}")